In [18]:
STEP_LIMIT = 1000

In [19]:
import gymnasium as gym
from gymnasium.wrappers import GrayScaleObservation, RecordEpisodeStatistics, ResizeObservation, TimeLimit

from pokerl.env.pokemonblue import PokemonBlueEnv
from pokerl.env.wrappers import (
    ObservationAddPokemonLevel,
    ObservationAddPosition,
    ObservationDict,
    RewardCheckpoint,
    RewardDecreasingNoChange,
    RewardDecreasingSteps,
    RewardHistoryToInfo,
    RewardIncreasingBadges,
    RewardIncreasingCapturePokemon,
    RewardIncreasingPokemonLevel,
    RewardIncreasingPositionExploration,
    StopAtPokemon,
    ppFlattenInfo,
)


def create_env(interactive=False) -> gym.Env:
    env = PokemonBlueEnv(
        interactive=interactive,
        save_state="game_start"
    )
    # Setting observation
    env = ResizeObservation(env, 128)
    # env = GrayScaleObservation(env)
    # env = ObservationDict(env)
    # env = ObservationAddPosition(env)
    # env = ObservationAddPokemonLevel(env)
    # Setting reward
    env = RewardDecreasingNoChange(env, .001)
    # env = RewardDecreasingSteps(env, .001)
    # env = RewardIncreasingBadges(env, 100)
    # env = RewardIncreasingCapturePokemon(env, 10)
    # env = RewardIncreasingPokemonLevel(env, 10)
    # env = RewardIncreasingPositionExploration(env, 1)
    # env = RewardHistoryToInfo(env)
    # Post processing
    env = RewardCheckpoint(env)
    env = TimeLimit(env, STEP_LIMIT)
    # env = ppFlattenInfo(env)
    return env

In [20]:
import torch
from stable_baselines3 import ppo
from stable_baselines3.common.env_util import make_vec_env

from pokerl.agent.tools import get_device

policy_kwargs = {"activation_fn": torch.nn.ReLU, "net_arch": {"pi": [256, 256, 256, 256], "vf": [256, 256, 256, 256]}}
env = create_env()

ppo = ppo.PPO(
    "CnnPolicy",
    env,
    device=get_device(),
    policy_kwargs=policy_kwargs,
    n_steps=STEP_LIMIT,
    batch_size=STEP_LIMIT,
    learning_rate=0.0003,
    verbose=1
)

Using privateuseone:0 device

Wrapping the env with a `Monitor` wrapper

Wrapping the env in a DummyVecEnv.

Wrapping the env in a VecTransposeImage.

In [21]:
ppo = ppo.learn(STEP_LIMIT*100, progress_bar=True)

LiveError: Only one live display may be active at once

In [ ]:
env = create_env(True)

In [ ]:
from matplotlib import pyplot as plt
from tqdm import tqdm

obs, _ = env.reset()
for _ in range(200):
    ppo.predict(obs)
    action, _ = ppo.predict(obs)
    obs, reward, _, _, _ = env.step(action)
    print(f"Reward is: {reward}")

Reward is: 0
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: 0
Reward is: 0
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: 0
Reward is: 0
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01
Reward is: -0.01


KeyboardInterrupt: 

In [ ]:
last_info = env.tick()
while True:
    info = env.tick()
    for k, v in info.items():
        if "tick" in k:
            continue
        if (last_info[k] != v).any():
            print(f"{k}: {v}")
            last_info = info
    pass

c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.tick to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.tick` for environment variables or `env.get_wrapper_attr('tick')` that will search the reminding wrappers.
  logger.warn(


position: [ 6  3 38]
position: [ 6  4 38]
position: [ 6  5 38]
position: [ 5  5 38]
position: [ 3  5 38]
position: [ 2  5 38]
position: [ 1  5 38]
position: [ 1  6 38]
position: [ 1  7 37]
position: [ 2  7 37]
position: [ 3  7 37]
position: [ 4  7 37]
position: [ 5  7 37]
position: [ 6  6 37]
position: [ 6  5 37]
position: [ 6  4 37]
position: [ 7  3 37]
position: [7 3 0]
position: [5 5 0]
position: [6 5 0]
position: [7 5 0]
position: [7 6 0]


KeyboardInterrupt: 

In [ ]:
obs, info = env.reset()

In [ ]:
info

{'tick': 0,
 'pokemon_level': array([0, 0, 0, 0, 0, 0]),
 'badges': array(0),
 'position': array([0, 0, 0]),
 'absolute_position': array([0, 0]),
 'owned_pokemon': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'rewardHistory': deque([], maxlen=10000)}

In [ ]:
env = PokemonBlueEnv(interactive=True)

In [ ]:
from tqdm import tqdm

observation, info = env.reset()
done = False
for i in tqdm(range(1000)):
    action, _states = ppo.predict(observation)
    observation, reward, truncated, terminated, info = env.step(action)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:11<00:00, 88.53it/s]


TypeError: '<' not supported between instances of 'int' and 'dict'

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 79       |
|    iterations      | 1        |
|    time_elapsed    | 25       |
|    total_timesteps | 2048     |
---------------------------------
